In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
home_dir = "../"

import os
import pandas as pd

In [2]:
snps_with_alfa_mane_short_df = pd.read_csv(home_dir+"data/temp_popufreq_data/snps_with_alfa_mane_genomicToProtMapping_short.tsv", sep="\t")
print(snps_with_alfa_mane_short_df.columns)
print(snps_with_alfa_mane_short_df.shape)
print(snps_with_alfa_mane_short_df["snp_id"].unique().shape[0])
print(snps_with_alfa_mane_short_df["gene_symbol"].unique().shape[0])
print(snps_with_alfa_mane_short_df["mane_refseq_prot"].unique().shape[0])

/tmp/ipykernel_771790/2262752280.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  snps_with_alfa_mane_short_df = pd.read_csv(home_dir+"data/temp_popufreq_data/snps_with_alfa_mane_genomicToProtMapping_short.tsv", sep="\t")


Index(['snp_id', 'spdi', 'snp_raw_genes', 'gene_symbol', 'gene_id',
       'mane_refseq_nuc', 'mane_refseq_prot', 'mane_status',
       'mane_grch38_chr_no', 'mane_chr_start', 'mane_chr_end',
       'mane_chr_strand', 'mane_protein_variants', 'spdi_genomic_variants',
       'genomic_to_prot_variants_mapping_source',
       'genomic_to_mapped_prot_variants', 'alfa_chrom_acc_version',
       'alfa_1indexed_chrom_pos', 'alfa_ref_allele', 'alfa_alt_alleles',
       'alfa_popu_counts'],
      dtype='object')
(1072966, 21)
1069839
16512
16543


In [10]:
certain_snps_df = snps_with_alfa_mane_short_df[snps_with_alfa_mane_short_df["genomic_to_prot_variants_mapping_source"]!="uncertain"]
uncertain_snps_df = snps_with_alfa_mane_short_df[snps_with_alfa_mane_short_df["genomic_to_prot_variants_mapping_source"]=="uncertain"]
print(certain_snps_df.shape, uncertain_snps_df.shape)
print("certain num of proteins", certain_snps_df["mane_refseq_prot"].unique().shape[0])
print("uncertain num of proteins", uncertain_snps_df["mane_refseq_prot"].unique().shape[0])

non_overlapping_prots_in_uncertain_set = set(uncertain_snps_df["mane_refseq_prot"].unique().tolist()) - set(certain_snps_df["mane_refseq_prot"].unique().tolist())
print(len(non_overlapping_prots_in_uncertain_set))
print(non_overlapping_prots_in_uncertain_set)

uncertain_snps_of_nonoverlapping_prots_df = uncertain_snps_df[uncertain_snps_df["mane_refseq_prot"].isin(list(non_overlapping_prots_in_uncertain_set))]
print(uncertain_snps_of_nonoverlapping_prots_df.shape)
uncertain_snps_of_nonoverlapping_prots_df

(1032488, 21) (40478, 21)
certain num of proteins 16542
uncertain num of proteins 13186
1
{'NP_001403.1'}
(1, 21)


,snp_id,spdi,snp_raw_genes,gene_symbol,gene_id,mane_refseq_nuc,mane_refseq_prot,mane_status,mane_grch38_chr_no,mane_chr_start,...,mane_chr_strand,mane_protein_variants,spdi_genomic_variants,genomic_to_prot_variants_mapping_source,genomic_to_mapped_prot_variants,alfa_chrom_acc_version,alfa_1indexed_chrom_pos,alfa_ref_allele,alfa_alt_alleles,alfa_popu_counts
1072958,rs142945049,"NC_000023.11:20132224:G:A,NC_000023.11:2013222...",EIF1AX:1964,EIF1AX,1964,NM_001412.4,NP_001403.1,MANE Select,X,20124525,...,-,NP_001403.1:p.Asp98Glu,"G>A,G>C",uncertain,NaN,NC_000023.11,20132225,G,"A,C","27504:7,0"


In [11]:
variants = []
for i, row in enumerate(certain_snps_df.itertuples(index=False)):
    prot_variants = row.genomic_to_mapped_prot_variants.split(",")
    alt_alleles = row.alfa_alt_alleles.split(",")

    wt_population, mt_poulations = int(row.alfa_popu_counts.split(":")[0]), row.alfa_popu_counts.split(":")[1].split(",")
    total_population = wt_population+sum(list(map(int, mt_poulations)))
    
    # the following condition should never satisfy
    if len(prot_variants) != len(alt_alleles):
        print(i, row)
        break

    for j, alt_allele in enumerate(alt_alleles):
        row_dict = row._asdict()
        row_dict["alfa_alt_allele"] = alt_allele
        row_dict["prot_variant"] = prot_variants[j]

        mt_population = int(mt_poulations[j])
        row_dict["wt_population"] = wt_population
        row_dict["mt_population"] = mt_population
        row_dict["wt_freq"] = wt_population/total_population # freq should be computed here, since we are decomposing multiple SNVs into separate independent SNVs.
        row_dict["mt_freq"] = mt_population/total_population

        variants.append(row_dict)
    # break


variants_df = pd.DataFrame(variants)
print(variants_df.shape)
variants_df

(1207088, 27)


,snp_id,spdi,snp_raw_genes,gene_symbol,gene_id,mane_refseq_nuc,mane_refseq_prot,mane_status,mane_grch38_chr_no,mane_chr_start,...,alfa_1indexed_chrom_pos,alfa_ref_allele,alfa_alt_alleles,alfa_popu_counts,alfa_alt_allele,prot_variant,wt_population,mt_population,wt_freq,mt_freq
0,rs1491003598,NC_000001.11:205269217:A:G,TMCC2:9911,TMCC2,9911,NM_014858.4,NP_055673.2,MANE Select,1,205227946,...,205269218,A,G,8988:1,G,NP_055673.2:p.Lys339Arg,8988,1,0.999889,0.000111
1,rs1436854664,NC_000001.11:205271827:T:A,TMCC2:9911,TMCC2,9911,NM_014858.4,NP_055673.2,MANE Select,1,205227946,...,205271828,T,A,14050:1,A,NP_055673.2:p.Cys612Ser,14050,1,0.999929,0.000071
2,rs1431352460,NC_000001.11:205241636:A:G,TMCC2:9911,TMCC2,9911,NM_014858.4,NP_055673.2,MANE Select,1,205227946,...,205241637,A,G,32062:1,G,NP_055673.2:p.Met114Val,32062,1,0.999969,0.000031
3,rs1429019865,NC_000001.11:205241751:T:C,TMCC2:9911,TMCC2,9911,NM_014858.4,NP_055673.2,MANE Select,1,205227946,...,205241752,T,C,4470:1,C,NP_055673.2:p.Leu152Pro,4470,1,0.999776,0.000224
4,rs1423575191,NC_000001.11:205269046:C:T,TMCC2:9911,TMCC2,9911,NM_014858.4,NP_055673.2,MANE Select,1,205227946,...,205269047,C,T,21382:2,T,NP_055673.2:p.Pro282Leu,21382,2,0.999906,0.000094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207083,rs33947415,"NC_000011.10:5226683:C:G,NC_000011.10:5226683:C:T",HBB:3043,HBB,3043,NM_000518.5,NP_000509.1,MANE Select,11,5225464,...,5226684,C,T,133516:146,T,NP_000509.1:p.Gly70Ser,133516,146,0.998908,0.001092
1207084,rs33945705,"NC_000011.10:5226671:C:A,NC_000011.10:5226671:...",HBB:3043,HBB,3043,NM_000518.5,NP_000509.1,MANE Select,11,5225464,...,5226672,C,T,20906:1,T,NP_000509.1:p.Asp74Asn,20906,1,0.999952,0.000048
1207085,rs33930165,"NC_000011.10:5227002:C:A,NC_000011.10:5227002:...",HBB:3043,HBB,3043,NM_000518.5,NP_000509.1,MANE Select,11,5225464,...,5227003,C,T,92220:69,T,NP_000509.1:p.Glu7Lys,92220,69,0.999252,0.000748
1207086,rs11549407,"NC_000011.10:5226773:G:A,NC_000011.10:5226773:...",HBB:3043,HBB,3043,NM_000518.5,NP_000509.1,MANE Select,11,5225464,...,5226774,G,A,134764:42,A,NP_000509.1:p.Gln40Ter,134764,42,0.999688,0.000312


In [12]:
from Bio.PDB.Polypeptide import protein_letters_3to1 # 20 amino acids: ACDEFGHIKLMNPQRSTVWY
def is_standard_protein_variant(row):
    # print(row)
    # raise
    v = row.prot_variant
    wt_aa = v.split(":")[1][2:5]
    mt_aa = v.split(":")[1][-3:]
    pos = int(v.split(":")[1][5:-3])

    if str.upper(wt_aa) not in protein_letters_3to1: return False
    if str.upper(mt_aa) not in protein_letters_3to1: return False
    return True

In [13]:
# apply filters
x = variants_df.copy()

# removing variants where mt-population count is 0
zero_mt_popu_count_removed = x[x["mt_population"].apply(lambda y: y!=0)]
print(zero_mt_popu_count_removed.shape[0], variants_df.shape[0]-zero_mt_popu_count_removed.shape[0])

# removing variants where alfa-alt-allle not in [A, C, G, T]
alfa_alt_allele_single_nucletide = zero_mt_popu_count_removed[zero_mt_popu_count_removed["alfa_alt_allele"].apply(lambda y: len(y)==1)]
print(alfa_alt_allele_single_nucletide.shape[0], zero_mt_popu_count_removed.shape[0]-alfa_alt_allele_single_nucletide.shape[0])

# removing variants where protein variants not in standard amino-acids
standard_prot_var = alfa_alt_allele_single_nucletide[alfa_alt_allele_single_nucletide.apply(is_standard_protein_variant, axis=1)]
print(standard_prot_var.shape[0], alfa_alt_allele_single_nucletide.shape[0]-standard_prot_var.shape[0])

cleaned_variants_df = standard_prot_var.copy()

1047930 159158
1047930 0
1039905 8025


In [14]:
non_standard_prot_var = alfa_alt_allele_single_nucletide[~alfa_alt_allele_single_nucletide.apply(is_standard_protein_variant, axis=1)]
non_standard_prot_var

,snp_id,spdi,snp_raw_genes,gene_symbol,gene_id,mane_refseq_nuc,mane_refseq_prot,mane_status,mane_grch38_chr_no,mane_chr_start,...,alfa_1indexed_chrom_pos,alfa_ref_allele,alfa_alt_alleles,alfa_popu_counts,alfa_alt_allele,prot_variant,wt_population,mt_population,wt_freq,mt_freq
206,rs369388803,"NC_000011.10:4388358:G:A,NC_000011.10:4388358:G:T",TRIM21:6737,TRIM21,6737,NM_003141.4,NP_003132.2,MANE Select,11,4384897,...,4388359,G,"A,T","38448:1,0",A,NP_003132.2:p.Gln226Ter,38448,1,0.999974,0.000026
236,rs1455280111,"NC_000019.10:54574902:G:A,NC_000019.10:5457490...",LILRA2:11027,LILRA2,11027,NM_001130917.3,NP_001124389.2,MANE Select,19,54573790,...,54574903,G,"A,T","23800:541,0",A,NP_001124389.2:p.Trp175Ter,23800,541,0.977774,0.022226
392,rs143337690,"NC_000019.10:54575514:G:A,NC_000019.10:5457551...",LILRA2:11027,LILRA2,11027,NM_001130917.3,NP_001124389.2,MANE Select,19,54573790,...,54575515,G,"A,C","44876:2,4",A,NP_001124389.2:p.Trp305Ter,44876,2,0.999866,0.000045
493,rs566724960,"NC_000016.10:21988740:G:A,NC_000016.10:2198874...",PDZD9:255762,PDZD9,255762,NM_001363519.1,NP_001350448.1,MANE Select,16,21983865,...,21988741,G,"A,C","20444:1,0",A,NP_001350448.1:p.Arg88Ter,20444,1,0.999951,0.000049
511,rs1384763791,"NC_000006.12:28085561:G:C,NC_000006.12:2808556...",ZNF165:7718,ZNF165,7718,NM_001376491.1,NP_001363420.1,MANE Select,6,28080568,...,28085562,G,"C,T","35430:0,1",T,NP_001363420.1:p.Glu28Ter,35430,1,0.999972,0.000028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1206871,rs1130385,"NC_000006.12:32664860:C:A,NC_000006.12:3266486...",HLA-DQB1:3119,HLA-DQB1,3119,NM_002123.5,NP_002114.3,MANE Select,6,32659467,...,32664861,C,A,16274:3851,A,NP_002114.3:p.Glu106Ter,16274,3851,0.808646,0.191354
1207022,rs377059218,"NC_000023.11:135965049:G:A,NC_000023.11:135965...",MMGT1:93380,MMGT1,93380,NM_173470.3,NP_775741.1,MANE Select,X,135960588,...,135965050,G,A,24904:3,A,NP_775741.1:p.Arg124Ter,24904,3,0.999880,0.000120
1207055,rs760563458,"NC_000024.10:9359388:G:A,NC_000024.10:9359388:G:T",TSPY8:728403,TSPY8,728403,NM_001243721.2,NP_001230650.1,MANE Select,Y,9357797,...,9359389,G,A,2794:12,A,NP_001230650.1:p.Trp237Ter,2794,12,0.995723,0.004277
1207073,rs33986703,"NC_000011.10:5226969:T:A,NC_000011.10:5226969:...",HBB:3043,HBB,3043,NM_000518.5,NP_000509.1,MANE Select,11,5225464,...,5226970,T,A,23408:1,A,NP_000509.1:p.Lys18Ter,23408,1,0.999957,0.000043


In [15]:
cleaned_variants_df.shape

(1039905, 27)

In [16]:
prot_variants = []
for i, row in enumerate(cleaned_variants_df.itertuples(index=False)):
    row_dict = row._asdict()
    v = row.prot_variant
    row_dict["prot_acc_version"]= v.split(":")[0]
    row_dict["wt_aa"] = v.split(":")[1][2:5]
    row_dict["mt_aa"] = v.split(":")[1][-3:]
    row_dict["1indexed_prot_mt_pos"] = int(v.split(":")[1][5:-3])
    prot_variants.append(row_dict)

prot_variants_df = pd.DataFrame(prot_variants)
prot_variants_df.shape

(1039905, 31)

In [17]:
cls_labeled_variants_df = prot_variants_df.copy()
cls_labeled_variants_df.loc[cls_labeled_variants_df["mt_freq"]>=.01, "class"] = "Common"
cls_labeled_variants_df.loc[(cls_labeled_variants_df["mt_freq"]<.01) & (cls_labeled_variants_df["mt_freq"]>=.001), "class"] = "Rare"
cls_labeled_variants_df.loc[(cls_labeled_variants_df["mt_freq"]<.001), "class"] = "Ultra-rare"
cls_labeled_variants_df.loc[cls_labeled_variants_df["mt_population"]==1, "class"] = "Singleton"
cls_labeled_variants_df.drop_duplicates(keep="first", inplace=True, ignore_index=True)
print(cls_labeled_variants_df["class"].value_counts())
print(cls_labeled_variants_df.shape)

Singleton     582632
Ultra-rare    394536
Rare           37026
Common         25711
Name: class, dtype: int64
(1039905, 32)


In [22]:
from models.aa_common.plots_helper import popu_freq_class_order
def print_summary(df, classes):
    print("", "#-genes", "#-proteins", "#-protein-variants", "#-unique-genomic-variants", sep="\t")
    for cls in classes:
        cls_df = df[df["class"]==cls]
        n_genes = cls_df["gene_symbol"].unique().shape[0]
        n_prots = cls_df["prot_acc_version"].unique().shape[0]
        n_prot_variants = cls_df["prot_variant"].shape[0] # this and the following line are same
        n_genomic_variants = cls_df[["alfa_chrom_acc_version", "alfa_1indexed_chrom_pos", "alfa_ref_allele", "alfa_alt_allele"]].drop_duplicates(keep="first").shape[0]
        print(cls, n_genes, n_prots, n_prot_variants, n_genomic_variants, sep="\t")

    total_n_genes = df["gene_symbol"].unique().shape[0]
    total_n_prots = df["prot_acc_version"].unique().shape[0]
    total_n_prot_variants = df["prot_variant"].shape[0]
    total_n_genomic_variants = df[["alfa_chrom_acc_version", "alfa_1indexed_chrom_pos", "alfa_ref_allele", "alfa_alt_allele"]].drop_duplicates(keep="first").shape[0]
    print("total", total_n_genes, total_n_prots, total_n_prot_variants, total_n_genomic_variants, sep="\t")

print_summary(cls_labeled_variants_df, popu_freq_class_order)

	#-genes	#-proteins	#-protein-variants	#-unique-genomic-variants
Singleton	16441	16472	582632	580876
Ultra-rare	16392	16423	394536	393340
Rare	12184	12202	37026	36953
Common	9500	9509	25711	25697
total	16511	16542	1039905	1036866


In [19]:
cls_labeled_variants_df.to_csv(home_dir+"data/temp_popufreq_data/cls_labeled_variants.tsv", index=False, sep="\t")